<a href="https://colab.research.google.com/github/Jnyambok/Building-Blocks-of-Mlops/blob/main/Feature_Store_and_Expectations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install -U 'hopsworks[python,great_expectations]' --quiet

In [2]:
import pandas as pd
trans_df = pd.read_csv("https://repo.hops.works/master/hopsworks-tutorials/data/card_fraud_data/transactions.csv", parse_dates=["datetime"])
trans_df.head(3)
profiles_df = pd.read_csv("https://repo.hops.works/master/hopsworks-tutorials/data/card_fraud_data/profiles.csv", parse_dates=["birthdate"])
profiles_df.head(3)

,name,sex,mail,birthdate,City,Country,cc_num
0,Catherine Zimmerman,F,valenciajason@hotmail.com,1988-09-20,Bryn Mawr-Skyway,US,4796807885357879
1,Michael Williams,M,brettkennedy@yahoo.com,1977-03-01,Gates-North Gates,US,4529266636192966
2,Jessica Krueger,F,marthacruz@hotmail.com,1947-09-10,Greenfield,US,4922690008243953


## <span style="color:#ff5f27;"> 💽 1. Loading the Data & Minimal Feature Engineering</span>

The data we will use comes from two different CSV files:

- `transactions.csv`: transaction information such as timestamp, location, and the amount. Importantly, the binary `fraud_label` variable tells us whether a transaction was fraudulent or not.
- `profiles.csv`: credit card user information such as birthdate and city of residence.

We can conceptualize these CSV files as originating from separate data sources.
**Both files have a credit card number column `cc_num` in common, which we can use for joins.**
This could for instance be the age of a customer at the time of a transaction, which combines the `birthdate` feature from `profiles.csv` with the `datetime` feature from `transactions.csv`.

In [3]:
import numpy as np

# Compute age at transaction.
age_df = trans_df.merge(profiles_df, on="cc_num", how="left")
trans_df["age_at_transaction"] = (age_df["datetime"] - age_df["birthdate"]) / np.timedelta64(365, "D")

# Convert date time object to unix epoch in milliseconds
trans_df.datetime = trans_df.datetime.values.astype(np.int64) // 10 ** 6

A [feature group](https://docs.hopsworks.ai/feature-store-api/latest/generated/feature_group/) can be seen as a collection of conceptually related features. In our case, we will create a feature group for the transaction data. It will have `cc_num` as primary key.

Before we can create a feature group we need to connect to our feature store.

In [6]:
import hopsworks
project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214661


To create a feature group we need to give it a name and specify a primary key. It is also good to provide a description of the contents of the feature group and a version number, if it is not defined it will automatically be incremented to `1`.

In [8]:
trans_fg = fs.get_or_create_feature_group(
    name="transactions",
    version=1,
    primary_key=["cc_num"],
    event_time="datetime"
)

At this point, we have only specified some metadata for the feature group. It does not store any data or even have a schema defined for the data. To make the feature group persistent we populate it with its associated data using the `insert` function. We are inserting a single row here.

In [9]:
job, _ = trans_fg.insert(trans_df.head(1))

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1214661/fs/1202290/fg/1411616


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: transactions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214661/jobs/named/transactions_1_offline_fg_materialization/executions


An Expectation Suite in Great Expectations contains multiple expectations which will be evaluated against a DataFrame on validation. This step will generate a report gathering information on whether the DataFrame fulfills those expectations. Setup the validation pipeline using the usual GE object and syntax. Once your expectation suite is ready, register it with the Hopsworks backend.

In [10]:
import great_expectations as ge
from great_expectations.core import ExpectationSuite, ExpectationConfiguration

When prototyping an expectation suite you can use the Great Expectations DataFrame wrapper to evaluate expectations directly and enable auto-complete.

In [11]:
ge_trans_df = ge.from_pandas(trans_df)

In [12]:
validation_result = ge_trans_df.expect_column_values_to_be_increasing(
    column="datetime",
    strictly=False
)
print(validation_result)

{
  "success": false,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_increasing",
    "kwargs": {
      "column": "datetime",
      "strictly": false,
      "result_format": "BASIC"
    },
    "meta": {}
  },
  "result": {
    "element_count": 106020,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 6030,
    "unexpected_percent": 5.68760611205433,
    "unexpected_percent_total": 5.68760611205433,
    "unexpected_percent_nonmissing": 5.68760611205433,
    "partial_unexpected_list": [
      1641371171000,
      1641477101000,
      1642384099000,
      1642523809000,
      1643931865000,
      1644163781000,
      1645852498000,
      1642536184000,
      1641365200000,
      1641424223000,
      1643133776000,
      1641611869000,
      1641855524000,
      1643878183000,
      1641241959000,
      1641401031000,
      1644986360000,
      1641767720000,
      1642210633000,
      1641428017000
    ]
  },
  "meta": {},
  "except

Note that this does not append the expectation to the suite, however the result contains all the necessary information to recreate the expectation.

In [18]:
expectation_suite_trans = ge_trans_df.get_expectation_suite()
expectation_suite_trans.expectation_suite_name = "transaction_suite"
print(expectation_suite_trans)

{
  "expectation_suite_name": "transaction_suite",
  "ge_cloud_id": null,
  "expectations": [],
  "data_asset_type": "Dataset",
  "meta": {
    "great_expectations_version": "0.18.12"
  }
}


In [19]:
expectation_suite_trans.add_expectation(validation_result["expectation_config"])
print(expectation_suite_trans)

{
  "expectation_suite_name": "transaction_suite",
  "ge_cloud_id": null,
  "expectations": [
    {
      "expectation_type": "expect_column_values_to_be_increasing",
      "kwargs": {
        "column": "datetime",
        "strictly": false,
        "result_format": "BASIC"
      },
      "meta": {}
    }
  ],
  "data_asset_type": "Dataset",
  "meta": {
    "great_expectations_version": "0.18.12"
  }
}


You can populate the suite with various expectation configurations. Many standard expectations are implemented in Great Expectations, from computing the mean of a column to counting the number of null values for example.

In [20]:
# Check for errors which could lead to technical issues

expectation_suite_trans.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_unique",
        kwargs={"column":"tid", "result_format":"COMPLETE"}
    )
)

expectation_suite_trans.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_in_set",
        kwargs={
            "column":"fraud_label",
            "value_set": [0,1]
        }
    )
)

# Assess data correctness

expectation_suite_trans.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"amount",
            "min_value": 0
        }
    )
)

expectation_suite_trans.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"age_at_transaction",
            "min_value": 17,
            "max_value": 130
        }
    )
)

# Or simply to monitor data statistics and quality, e.g number of null values

expectation_suite_trans.add_expectation(
    ExpectationConfiguration(
      expectation_type="expect_column_mean_to_be_between",
      kwargs={"column":"age_at_transaction", "min_value": 55, "max_value": 70}
    )
)

expectation_suite_trans.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_not_be_null",
        kwargs={"column":"category"}
    )
)


{"expectation_type": "expect_column_values_to_not_be_null", "kwargs": {"column": "category"}, "meta": {}}

You can validate and go check your result

In [21]:
ge_trans_df = ge.from_pandas(trans_df, expectation_suite=expectation_suite_trans)

validation_report_trans = ge_trans_df.validate()

For complex DataFrames, Great Expectations offers a profiler which generates a basic expectation suite tailored to your data. You can then use this suite as you would any other expectation suite with Hopsworks.

In [23]:
# The "ALWAYS" ingestion policy inserts data even when validation fails,
# ideal to avoid data loss and rapid prototyping
trans_fg.save_expectation_suite(expectation_suite_trans, validation_ingestion_policy="ALWAYS")

Attached expectation suite to Feature Group, edit it at https://c.app.hopsworks.ai:443/p/1214661/fs/1202290/fg/1411616


Once the suite is registered in the backend, data validation will run on every insert without additional boilerplate. The suite is retrieved from the backend, used to validate the DataFrame and the resulting validation report uploaded. Depending on the ingestion policy and validation success, data are subsequently inserted in the Feature Group. The example below illustrate the "ALWAYS" use case where insertion is performed despite a validation failure.

In [24]:
trans_job, trans_report = trans_fg.insert(trans_df)

Validation failed.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1214661/fs/1202290/fg/1411616


Uploading Dataframe: 100.00% |██████████| Rows 106020/106020 | Elapsed Time: 00:05 | Remaining Time: 00:00


Launching job: transactions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214661/jobs/named/transactions_1_offline_fg_materialization/executions


The "STRICTLY" ingestion policy aborts insertion job if the validation fails. However it still uploads the report to Hopsworks so you can check the UI to understand what went wrong and have a record of your attempt.

In [27]:
# Find the culprit (easier done through the UI)
culprit = [result for result in trans_report.results if result.success is False][0]
print(f"culprit is expectation {culprit.expectation_config['meta']['expectationId']}, with type {culprit.expectation_config['expectation_type']} applied to column {culprit.expectation_config['kwargs']['column']}")

culprit is expectation 700436, with type expect_column_values_to_be_increasing applied to column datetime


In [29]:
# We need to sort the datetime column to fix it
trans_df.sort_values(by=["datetime"], inplace=True)

trans_job, trans_report = trans_fg.insert(trans_df)
#Check the UI

Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1214661/fs/1202290/fg/1411616


Uploading Dataframe: 100.00% |██████████| Rows 106020/106020 | Elapsed Time: 00:07 | Remaining Time: 00:00
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/data_science/Resources/jobs/transactions_1_offline_fg_materialization/config_1741707244296) to trigger the materialization job again.

